# FDSN-Mass Downloader
## Rectangular Selection

obspy.clients.fdsn package contains a client to acces web servers that implement the FDSN web service definitions. Here are the step for downloading the seismic data:

1. specify the geographical region from which to download data;
2. define a number of other restrictions (temporal, data quality,...);
3. launch the download;

The mass downloader module will acquire all waveforms and associated station information across all known FDSN web service implementations producing a clean data set ready for further use. It works by:

a. figuring out what stations each provider offers;
b. downloading MiniSEED and associated StationXML meta information in an efficient and data center friendly manner, and
c. dealing with all the nasty real-world data issues like missing or incomplete data, duplicate data across data centers, e.g.

Basic optional automatic quality control by assuring that the data has no-gaps/overlaps or is available for a certain percentage of the requested time span.

It can relaunch download to acquire missing pieces which might happen for example if a data center has been offline.

It can assure that there always is a corresponding StationXML file for the waveforms.

The classic seismological data set consists of waveform recordings for a certain earthquake. This example downloads a day series data of NZ network (New Zealand National Seismograph Network) with the network code start with Z and the star indicates of any network code that start with Z, the starttime is UTCDateTime 2023225T600 and endtime UTCDateTime(2023226t600). It will furthermore only download data with an epicentral distance between longitude 127.40000 and 128.40000 degrees and latitude between 0.95 and 1.20 degrees and some additional restrictions. Be aware that this example will attempt to download data from all FDSN data centers that ObsPy knows of and combine it into one data set.

In [ ]:
import obspy
from obspy.clients.fdsn.mass_downloader import RectangularDomain, \
    Restrictions, MassDownloader
from obspy import UTCDateTime

domain = RectangularDomain(minlatitude=0.95, maxlatitude=1.20,
                           minlongitude=127.40000, maxlongitude=128.40000)

restrict = Restrictions(
    starttime=UTCDateTime(2023, 2, 25, 6, 0, 0),
    endtime=UTCDateTime(2023, 2, 26, 6, 0, 0), 
    network="NZ", station="Z*", location="10", channel="HH*",
    reject_channels_with_gaps=True,
    minimum_length=0.95,
    minimum_interstation_distance_in_m=1000)
mdl =  MassDownloader()
mdl.download(domain, restrict, mseed_storage="waveforms", 
         stationxml_storage="stations")

# Data set selection 
serves the purpose to limit the data to be downloaded to data useful for the purpose at hand. It is handled by two objects: subclasses of the Domain object and the Restrictions class.

The domain module currently defines three different domain types used to limit the geographical extent of the queried data: RectangularDomain, CircularDomain, and GlobalDomain. Subclassing Domain enables the construction of arbitrarily complex domains. Instances of these classes will later be passed to the function sparking the downloading process. 
Additional restrictions like temporal bounds, SEED identifier wildcards, and other things are set with the help of the Restrictions class are here:

1. starttime (UTCDateTime) – The start time of the data to be downloaded.
2. endtime (UTCDateTime) – The end time of the data.
3. network (str) – The network code. Can contain wildcards.
4. station (str) – The station code. Can contain wildcards.
5. location (str) – The location code. Can contain wildcards.
6. channel (str) – The channel code. Can contain wildcards.
7. reject_channels_with_gaps (bool) – If True (default), MiniSEED files with gaps and/or overlaps will be rejected.
8. minimum_length (float) – The minimum length of the data as a fraction of the requested time frame. After a channel has been downloaded it will be checked that its total length is at least that fraction of the requested time span. Will be rejected otherwise. Must be between 0.0 and 1.0, defaults to 0.9.
9. minimum_interstation_distance_in_m (float) – The minimum inter-station distance. Data from any new station closer to any existing station will not be downloaded. Also used for duplicate station detection as sometimes stations have different names for different webservice providers. Defaults to 1000 m.

# Storage Options
After determining what to download, the helpers must know where to store the requested data.